In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from itertools import chain
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import numpy as np
from numpy import random
from selenium.webdriver.chrome.options import Options
import pickle
import re
import socket
import pandas as pd
import requests

In [2]:
all_countries=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/all_countries.csv")
Product_Code=pd.read_csv("C:/Users/homa/Python-Notbook/TradeMap/Product_Codem.csv")
all_countries.head(2)

,Country,Unnamed: 1,Country1,Unnamed: 3
0,Afghanistan,NaN,Afghanistan,NaN
1,Albania,NaN,Albania,NaN


In [3]:



# لیست user-agent ها
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.6668.70 Safari/537.36',
]




def check_tor_ip():
    try:
        # ارسال درخواست به یک سرویس نمایش آی‌پی از طریق پروکسی Tor
        proxies = {
            'http': 'socks5://127.0.0.1:9050',
            'https': 'socks5://127.0.0.1:9050'
        }
        
        response = requests.get('https://icanhazip.com/', proxies=proxies, timeout=10)
        if response.status_code == 200:
            current_ip = response.text.strip()
            print(f"Current Tor IP: {current_ip}")
            return True
        else:
            print("Failed to retrieve IP via Tor.")
            return False
    except requests.RequestException as e:
        print(f"Error checking Tor IP: {e}")
        return False
 



# تابع تغییر آی‌پی Tor
def change_tor_ip():
    try:
        # اتصال به ControlPort Tor
        with socket.create_connection(('127.0.0.1', 9051)) as sock:
            # ارسال دستور احراز هویت با رمز عبور
            sock.send(b'AUTHENTICATE "Aa@97019701"\r\n')
            response = sock.recv(1024).decode("utf-8")
            
            if response.startswith('250'):
                # ارسال سیگنال NEWNYM برای تغییر آی‌پی
                sock.send(b'SIGNAL NEWNYM\r\n')
                response = sock.recv(1024).decode("utf-8")
                if response.startswith('250'):
                    if check_tor_ip():
                        print("Tor IP changed successfully.")
                    else:
                        change_tor_ip()
                
                else:
                    print("Failed to change Tor IP.")
            else:
                print("Authentication failed.")
    except Exception as e:
        print(f"Error changing Tor IP: {e}")











# تنظیم Selenium برای استفاده از Tor
def setup_driver():
    chrome_options = Options()

    # انتخاب تصادفی user-agent
    user_agent = random.choice(user_agents)
    chrome_options.add_argument(f"user-agent={user_agent}")
    
    # تنظیم پروکسی Tor
    chrome_options.add_argument('--proxy-server=socks5://127.0.0.1:9050')
    
    # راه‌اندازی WebDriver
    driver = webdriver.Chrome(options=chrome_options)
    
    return driver






df = pd.DataFrame()
temp = pd.DataFrame()
driver = setup_driver()  # ایجاد WebDriver

waits= WebDriverWait(driver, 30)
try: 
    change_tor_ip()
    time.sleep(random.randint(5,15))

    driver.get("https://www.trademap.org")
except:
    driver.get("https://www.trademap.org")

time.sleep(30)
   
for i in range(len(Product_Code)):
    
    change_tor_ip()  # تغییر آی‌پی Tor برای هر درخواست جدید
    time.sleep(random.randint(5,15))  # زمان دادن به Tor برای تغییر آی‌پی

    #-------------------------------------------------------------------# عملیات لاگین کاربر
    try:
        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
        if logintext.text=='Login': 
                
            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
            login.click()
            url= driver.current_url
            if url!='https://www.trademap.org/Index.aspx': 
                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                getemail.send_keys('homa.abedinzadeh@gmail.com')
                time.sleep(random.randint(5,20))
                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                getpassword.send_keys('Aa@97019701')
                time.sleep(random.randint(5,20))
                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
            
    
    except:
        print  ('LOGIN Error DAD')
    #-------------------------------------------------------------------# پایان

    change_tor_ip()
    time.sleep(random.randint(5,15))

    #-----------------------------------------------------------------انتخاب محصول
                    
    inputproduct = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
    proclick = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_c0'))).click()
    btn = waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()

    #---------------------------------------------------------------پایان

    #change_tor_ip()
    #time.sleep(random.randint(5,15))

    #--------------------------------------------------------------تعداد کشورهای داخل dropdown
    dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))  # Replace with actual ID     
    select = Select(dropdown_element)
    options = select.options
    #---------------------------------------------------------------پایان


    for j in range(len(options)):  
            try:
                

                #------------------------ انتخاب کشور از منوی کشویی
                dropdown_element = waits.until(EC.visibility_of_element_located((By.ID, 'ctl00_NavigationControl_DropDownList_Country')))
                select = Select(dropdown_element)
                options = select.options
                option_text = options[j].text

                if option_text != 'All':
                    select.select_by_visible_text(option_text)

#---------------------------------------------------------------------                        
                    #change_tor_ip()
                    #time.sleep(random.randint(5,15))
#---------------------------------------------------------------------


                    # تغییر تعداد رکوردها به 300
                    incshow = waits.until(EC.element_to_be_clickable((By.XPATH, "//select[@id='ctl00_PageContent_GridViewPanelControl_DropDownList_PageSize']")))
                    drop = Select(incshow)
                    drop.select_by_value('300')





                    # استخراج جدول
                    all_tables = pd.read_html(driver.page_source, attrs={'id': 'ctl00_PageContent_MyGridView1'})
                    temp = all_tables[0]

                    if temp.shape[0] != 1:
                        temp = temp.iloc[1:, 1:].iloc[:, :-4]
                        new_header = temp.iloc[0]
                        temp = temp[1:]
                        temp.columns = new_header

                        # افزودن اطلاعات کشور و کد محصول
                        temp['Importer'] = option_text
                        temp['productcode'] = re.sub(r'[^\w]', ' ', Product_Code['ProductCode'].iloc[i])
                        temp.drop(temp[temp['Exporters'] == 'Total'].index, inplace=True)

                        # تغییر نام ستون‌ها
                        average = temp.columns[15]
                        sharein = temp.columns[3]
                        temp.rename(columns={average: 'Average tariff%', sharein: 'ShareinCounryImports'}, inplace=True)

                        # افزودن به DataFrame اصلی
                        df = pd.concat([df, temp], axis=0)
                        temp = pd.DataFrame()

                        print(f"Selected option: {option_text}, {j}")


#---------------------------------------------------------------------                        
                        #change_tor_ip()
                        #time.sleep(random.randint(5,15))
#---------------------------------------------------------------------  

            except Exception as e:
                print(f"Error selecting option {j}: {e}")
                site = driver.current_url
                if site == 'https://www.trademap.org/Index.aspx':


#---------------------------------------------------------------------                        
                    
                    change_tor_ip()
                    time.sleep(random.randint(5,15))

#---------------------------------------------------------------------  

                        #-------------------------------------------------------------------# عملیات لاگین کاربر
                    try:
                        try: 
                            #text=driver.find_element(By.CLASS_NAME,"imglogo")
                            #if text != "imglogo":
         #---------------------------------------------------------------------                        
                    
                            change_tor_ip()
                            time.sleep(random.randint(5,15))

        #---------------------------------------------------------------------  
                            driver.get("https://www.trademap.org")
                        except:
                            driver.get("https://www.trademap.org")

                        logintext=driver.find_element(By.CSS_SELECTOR ,("#ctl00_MenuControl_Label_Login"))
                        if logintext.text=='Login': 
                                
                            login=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_MenuControl_Label_Login')))
                            login.click()
                            url= driver.current_url
                            if url!='https://www.trademap.org/Index.aspx': 
                                getemail= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Username')))
                                getemail.send_keys('homa.abedinzadeh@gmail.com')
                                time.sleep(random.randint(5,20))
                                getpassword= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#Password')))
                                getpassword.send_keys('Aa@97019701')
                                time.sleep(random.randint(5,20))
                                waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,'body > div.container > div > div:nth-child(2) > div > div > div > form > fieldset > div:nth-child(4) > div > button'))).click()
                            
                    
                    except:
                        print  ('LOGIN Error DAD')
                    #-------------------------------------------------------------------# پایان


                    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_deleteProduct'))).click()
                    waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Image_DeleteCountry'))).click()
                    inputproduct= waits.until(EC.element_to_be_clickable ((By.CSS_SELECTOR, '#ctl00_PageContent_RadComboBox_Product_Input')))
                    inputproduct.send_keys(Product_Code['ProductCode'].iloc[i])
                    clkinput= waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR,  '#ctl00_PageContent_RadComboBox_Product_c0'))).click()

                    btn=waits.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ctl00_PageContent_Button_TradeIndicators'))).click()
        
    waits.until(EC.element_to_be_clickable((By.ID, 'ctl00_MenuControl_Label_Menu_Home'))).click()




Current Tor IP: 192.42.116.211
Tor IP changed successfully.
